# Beleg2 - Multivariate lineare Regression

Implementieren Sie die multivariate lineare Regression mit Python und numpy in einem ipython notebook.
Hinweis: Ihre Lösung der multivariaten linearen Regression sollte auf numpy basieren und im Prinzip mit beliebig vielen Features zurechtkommen. D.h. es muss eine vektorielle Implementierung vorgenommen werden.

Benutzen Sie zur Lösung eine Kostenfunktion und das Gradientenabstiegsverfahren. Beachten Sie dabei folgende Punkte:

## Aufgabe 1: Erstellen Sie zuerst zum Testen Ihrer Lösung künstliche Datenwerte für zwei Merkmale (Features): 

#### X soll dabei eine Datenmatrix mit zwei Spalten sein, wobei die Werte zufällig aus einer Gleichverteilung (konstante Wahrscheinlichkeitsdichte in einem Intervall) gezogen werden.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
### Aufgabe 1
#Interval low-high
low = 0
high = 20
data = 100 #Anzahl der Datensätze
merkmale = 2 #Anzahl Spalten

#Creates a matrix with normal distributed values 
myX = np.random.uniform(low,high, (data,merkmale)) #Matrix mit 2 Spalten

original_X = myX.copy() #backup original matrix

#Inserts in matrix myX, at index 0, a vector of ones along axis 1
myX = np.insert(myX, 0, values=1, axis=1)
print("Matrix: ")
myX

## Aufgabe 2: Implementieren Sie die Hypothese (lineares Modell) als Python Funktion: linear_hypothesis(theta)

#### Die Pythonfunktion soll dabei eine Funktion zurückgeben:

In [ ]:
#calculates linear hypothesis
#Parameter: array of thetas

def linear_hypothesis(theta):
    return lambda X: X.dot(theta) #Function works for any given Matrix X

## Aufgabe 3: Generierung, Plotten und Feature Scaling

###  a) Nutzen Sie die Funktion linear_hypothesis(theta) zum Generieren künstlicher y-Werte (Zielwerte) für Ihre Merkmalsvektoren (Zeilen von X).  Addieren Sie zusätzich ein gaussches Rauschen auf die einzelnen y-Werte.

In [ ]:
#theta array
theta = np.array([1., 1., 1.]) 

#call function linear_hypothesis
h = linear_hypothesis(theta)

#Creates y values with function linear_hypothesis
y = h(myX)

print("Y values without Gaussian error: ")
print(y) #len(y) == data

#Defining mu and sigma for gaussian error
mu = 0.0
sigma = 1.5 

#Adds a Gaussian error to the values
y = y + np.random.normal(mu, sigma, data)

print("\nY values with Gaussian error: ")
print(y)


### b) Stellen Sie die X1-X2-Y Werte in einem 3D Plot dar, siehe: http://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html

Tutorial von O Really:
https://www.oreilly.com/learning/three-dimensional-plotting-in-matplotlib


In [ ]:
#Plotting    
x1 = myX[:,1]
x2 = myX[:,2]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


ax.scatter(x1, x2, y, c='r', marker='o')

ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('Y')
plt.title("Data plot")
plt.show()

### c) Implementieren Sie das Feature Scaling um neue x' Werte zu berechnen

In [ ]:
#The function receives a vector of values and scales them in a range 0-1
def feature_scale(vector):
    if(np.std(vector) != 0):
        vector = (vector-np.mean(vector))/np.std(vector)
    return vector


In [ ]:
#scales a matrix 
def scale_matrix(X):
    #skips first column of ones
    #for i in range(1,merkmale+1):
        #col = X[:,i]
        #X[:,i] = feature_scale(col)
    X = np.apply_along_axis(feature_scale, 0, X)
    return X

In [ ]:
myX = scale_matrix(myX)
myX

In [ ]:
myX[1].std()

In [ ]:
myX.T

In [ ]:
arr = np.array([1.,1.,0.5,0.1,-0.4,-0.9,-1])

np.ceil(arr.mean())

## Aufgabe 4: Implementieren Sie die Kostenfunktion J als Python Funktion: cost_function(x, y)
#### Die Pythonfunktion soll dabei eine Funktion zurückgeben, die die denParametervektor theta aufnimmt.
- j = cost_function(X, y) 
- print j(theta)
- 41.20 # Wert abhaengig von X und y


In [ ]:
# Aufgabe 4
#given a matrix and hypothesis values (y values) calculate the cost function
def cost_function(X,y):
    m = data
    return lambda theta: 1./(2*m)*((linear_hypothesis(theta)(X)-y)**2).sum()


j = cost_function(myX,y) #y values not scaled
print(j(theta)) #theta=[1,1,1]

##### das ist die Kostenfuntkion mit skalierten y-Werten, mit theta=1,1,1 kommt 0.0 als Ergebnis raus!!!!!
#thetas = [1.2,2,4]
#jj = cost_function(myX,y_scaled)
#print(jj(thetas))

## Aufgabe 5: Implementieren Sie das Gradientenabstiegsverfahren unter Benutzung der Kostenfunktion und der linearen Hypothese.

## 5a) Schreiben Sie eine Funktion die die Update Rules anwendet zur Berechnung der neuen theta-Werte:
theta = compute_new_theta(x, y, theta, alpha)


In [ ]:
def compute_new_theta(X,y,theta,alpha):
    m = data
    new_thetas = theta-alpha*(1.0/m)*np.dot(X.T,((linear_hypothesis(theta)(X))-y))
    return new_thetas

#Test method call with alpha 0.02 or 0.1
print(compute_new_theta(myX,y,theta, 0.02))    
print(compute_new_theta(myX,y,theta, 0.1))  

## 5b) Wählen Sie Startwerte in der Umgebung des Miniums der Kostenfunktion für theta. 
#Wenden Sie iterativ die compute_new_theta Funktion an und finden Sie so ein Theta mit niedrigen Kosten.
Kapseln Sie dies in eine Funktion:
gradient_descent(alpha, theta, nb_iterations, X, y)

In [ ]:
def gradient_descent(alpha, theta, nb_iterations, X, y):
    #Stores costs for each iteration
    costs = [] #cost = np.zeros(nb_iterations)
       
    #stores changes in theta for each iteration
    temp_thetas = np.matrix(np.zeros(myX.shape))
    #print(temp_thetas)
    
    #starting cost value
    j = cost_function(X,y)(theta)
    
    costs.append(j)   
    
    #Iterates all nb_iterations
    for i in range(nb_iterations-1):
        #compute new thetas
        temp_thetas = compute_new_theta(X,y,theta,alpha)
        #update
        theta = temp_thetas
        #new cost function is computed   
        j = cost_function(X,y)
        cost = j(theta)
        #new cost appended
        costs.append(cost)

    return {'theta': theta, "costs": costs, "last": costs[len(costs)-1]}


In [ ]:
iterations = 600
alpha = 0.01
theta = [0.,0.,0.]
gr1 = gradient_descent(alpha, theta, iterations, myX, y)
computed_theta = gr1['theta']
#print(gr1['costs'])#last cost

## 5c) Plotten Sie den Fortschritt (Verringerung der Kosten über den Iterationen) für 5b

In [ ]:
# Aufgabe 5c
import matplotlib.pyplot as plt
fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
plt.plot(gr1['costs'])
plt.show()

# 6) Stellen Sie die gefundene Hyperebene in einem 3D Plot zusammen mit den Daten dar

In [ ]:
#Retrieves column X1 and X2 from original matrix myX
X1 =myX[:,[1,]].flatten()#selection of all rows. Of each row then the first column
X2 =myX[:,[2,]].flatten()

#Creates a meshgrid of the matrix columns X1 and X2 
X1mesh, X2mesh = np.meshgrid(X1,X2)
matrixMeshed = np.column_stack((X1mesh.ravel(),X2mesh.ravel()))
matrixMeshed = np.insert(matrixMeshed, 0, values=1, axis=1) #füge 1 als erste Spalte ein

#Values for Z calculated with linear_hypothesis on the last computed thetas (gr1 above)
yvalues = np.apply_along_axis(linear_hypothesis(computed_theta), 1, matrixMeshed) 
yvalues = yvalues.reshape(X1mesh.shape) #yvalues gets the shape of X1mesh (data,data)


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

%matplotlib notebook
#ax.plot_wireframe(X1mesh, X2mesh, yvalues, rstride=4, cstride=4, alpha=0.4)
ax.plot_surface(X1mesh, X2mesh , yvalues, cmap='Oranges_r', alpha=0.01)
ax.scatter(X1,X2,y, c='r')
plt.title("3D Plot")
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('y')
plt.show()

## Test mit mehreren Features 

In [ ]:
low = 0
high = 50
cols = 100 #features
row = 50
grossX = np.random.uniform(low,high, (row,cols))
grossX = np.insert(grossX, 0, values=1, axis=1)
#print(grossX)
thetas = np.ones(cols+1)
h = linear_hypothesis(thetas)
y_val = h(grossX)
y_val = y_val +np.random.normal(0, 1.5, row)

iterations = 2500
alpha = 0.00002
grr = gradient_descent(alpha, thetas, iterations, grossX, y_val)
computed_theta = grr['theta']

fig = plt.figure()
plt.plot(grr['costs'])
plt.show()